Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [1]:
let name = "Vimarsh Sathia"
let rollno = "CS17B046"

val name : string = "Vimarsh Sathia"


val rollno : string = "CS17B046"


## Important notes about grading:

1. **Compiler errors:** All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please visit consulting hours. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. **Late assignments:** Please carefully review the course website's policy on late assignments, as all assignments handed in after the deadline will be considered late. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

## A Prolog interpreter in OCaml

In this assignment, you will implement a Prolog interpreter in OCaml with support for backtracking (recover from bad choices) and choice points (produce multiple results).

We will start with the definition of terms, predicates, clauses, goals and programs.

In [2]:
type constant = Constant of string
type variable = Variable of string
type func = Function of string * term list
and term = C of constant | V of variable | F of func

type predicate = Predicate of string * term list

type head = predicate
type body = predicate list

type clause = Fact of head | Rule of head * body

type goal = predicate list

type program = clause list

type constant = Constant of string


type variable = Variable of string


type func = Function of string * term list
and term = C of constant | V of variable | F of func


type predicate = Predicate of string * term list


type head = predicate


type body = predicate list


type clause = Fact of head | Rule of head * body


type goal = predicate list


type program = clause list


The following stringification functions should help you debug the interpreter. 

In [3]:
let rec string_of_f_list f tl =
  let _, s = List.fold_left (fun (first, s) t -> 
    let prefix = if first then "" else s ^ ", " in
    false, prefix ^ (f t)) (true,"") tl
  in 
  s

let rec string_of_term t =
  match t with
  | C (Constant c) -> c
  | V (Variable v) -> v
  | F (Function (f,tl)) -> 
      f ^ "(" ^ (string_of_f_list string_of_term tl) ^ ")"

let string_of_predicate (Predicate (p,tl)) =
  p ^ "(" ^ (string_of_f_list string_of_term tl) ^")"
  
let string_of_predicate_list pl =
  string_of_f_list string_of_predicate pl
  
let string_of_goal g =
  "?- " ^ (string_of_predicate_list g)

let string_of_clause c =
  match c with
  | Fact f -> string_of_predicate f ^ "."
  | Rule (h,b) -> string_of_predicate h ^ " :- " ^ (string_of_predicate_list b) ^ "."

let string_of_program p =
  List.fold_left (fun acc c -> acc ^ (string_of_clause c) ^ "\n") "" p

val string_of_f_list : ('a -> string) -> 'a list -> string = <fun>


val string_of_term : term -> string = <fun>


val string_of_predicate : predicate -> string = <fun>


val string_of_predicate_list : predicate list -> string = <fun>


val string_of_goal : predicate list -> string = <fun>


val string_of_clause : clause -> string = <fun>


val string_of_program : clause list -> string = <fun>


Below are more helper functions:

In [4]:
let var v = Variable v
let var_t v = V (var v)
let const c = Constant c
let const_t c = C (Constant c)
let func f l = Function (f,l)
let func_t f l = F (func f l)
let pred p l = Predicate (p,l)
let fact f = Fact f
let rule h b = Rule (h,b)

val var : string -> variable = <fun>


val var_t : string -> term = <fun>


val const : string -> constant = <fun>


val const_t : string -> term = <fun>


val func : string -> term list -> func = <fun>


val func_t : string -> term list -> term = <fun>


val pred : string -> term list -> predicate = <fun>


val fact : head -> clause = <fun>


val rule : head -> body -> clause = <fun>


## Problem 1

In this problem, you will implement the function:

```ocaml
occurs_check : variable -> term -> bool
```

`occurs_check v t` returns `true` if the variable `v` occurs in `t`. 

In [5]:
let rec occurs_check v t = 
  match t with
  |C c -> false
  |V x -> if(v=x)then true else false
  |F (Function(f,tl)) -> List.fold_left (fun acc x -> acc || occurs_check v x) (false) (tl);;

val occurs_check : variable -> term -> bool = <fun>


In [6]:
assert (occurs_check (var "X") (var_t "X"));;
assert (not (occurs_check (var "X") (var_t "Y")));;
assert (occurs_check (var "X") (func_t "f" [var_t "X"]))

- : unit = ()


- : unit = ()


- : unit = ()


## Problem 2

Implement the following functions which return the variables contained in them:

```ocaml
variables_of_term      : term -> VarSet.t
variables_of_predicate : predicate -> VarSet.t
variables_of_clause    : clause -> VarSet.t
```

In [7]:
module VarSet = Set.Make(struct type t = variable let compare = Pervasives.compare end)
(* API Docs for Set : https://caml.inria.fr/pub/docs/manual-ocaml/libref/Set.S.html *)

let rec variables_of_term t = 
  let s = VarSet.empty in 
    match t with
    |C c -> s
    |V v -> VarSet.add v s
    |F (Function(f,tl)) -> List.fold_left (fun acc x -> VarSet.union (acc) (variables_of_term x)) s tl;;
  
let rec variables_of_predicate (Predicate (_, l)) =
  (* l is a term list *)
   List.fold_left (fun acc x -> VarSet.union (acc) (variables_of_term x)) VarSet.empty l;;
  
let variables_of_clause c =
match c with
|Fact p -> variables_of_predicate p
|Rule (p,pl) -> 
  let l = List.fold_left (fun acc p -> VarSet.union (acc) (variables_of_predicate p)) VarSet.empty pl in
  VarSet.union (variables_of_predicate p) l;;

module VarSet :
  sig
    type elt = variable
    type t
    val empty : t
    val is_empty : t -> bool
    val mem : elt -> t -> bool
    val add : elt -> t -> t
    val singleton : elt -> t
    val remove : elt -> t -> t
    val union : t -> t -> t
    val inter : t -> t -> t
    val diff : t -> t -> t
    val compare : t -> t -> int
    val equal : t -> t -> bool
    val subset : t -> t -> bool
    val iter : (elt -> unit) -> t -> unit
    val map : (elt -> elt) -> t -> t
    val fold : (elt -> 'a -> 'a) -> t -> 'a -> 'a
    val for_all : (elt -> bool) -> t -> bool
    val exists : (elt -> bool) -> t -> bool
    val filter : (elt -> bool) -> t -> t
    val partition : (elt -> bool) -> t -> t * t
    val cardinal : t -> int
    val elements : t -> elt list
    val min_elt : t -> elt
    val min_elt_opt : t -> elt option
    val max_elt : t -> elt
    val max_elt_opt : t -> elt option
    val choose : t -> elt
    val choose_opt : t -> elt option
    val split : elt -> t -> t * bool *

val variables_of_term : term -> VarSet.t = <fun>


val variables_of_predicate : predicate -> VarSet.t = <fun>


val variables_of_clause : clause -> VarSet.t = <fun>


In [8]:
assert (variables_of_term (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);;
assert (variables_of_predicate (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);;
assert (variables_of_clause (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        VarSet.of_list [var "X"; var "Y"]);;
assert (variables_of_clause (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]])= 
        VarSet.of_list [var "X"; var "Y"])

- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


## Problem 3

The value of type `term Substitution.t` is a OCaml map whose key is of type `variable` and value is of type `term`. In other words, it is map from variables to terms. Implement substitution functions that takes a `term Substitution.t` and uses that to perform the substitution:

```ocaml
substitute_in_term : term Substitution.t -> term -> term
substitute_in_predicate : term Substitution.t -> predicate -> predicate
substitute_in_clause : term Substitution.t -> clause -> clause
```

In [9]:
module Substitution = Map.Make(struct type t = variable let compare = Pervasives.compare end)
(* See API docs for OCaml Map: https://caml.inria.fr/pub/docs/manual-ocaml/libref/Map.S.html *)

let string_of_substitution s =
  "{" ^ (Substitution.fold (fun (Variable v) t s -> s ^ "; " ^ v ^ " -> " ^ (string_of_term t)) s "") ^ "}"

let rec substitute_in_term s t =
  match t with
  |C c -> t
  |V x -> if(Substitution.mem x s)then Substitution.find x s else t
  |F (Function (f,tl)) -> 
    F(Function(f,List.fold_left (fun acc x -> acc @ [(substitute_in_term s x)] ) [] tl));;
  
let rec substitute_in_predicate s (Predicate (p,l)) =
  Predicate (p,List.fold_left (fun acc x -> acc @ [(substitute_in_term s x)]) [] l);;
  
let substitute_in_clause s c =
  match c with
  |Fact p -> Fact (substitute_in_predicate s p)
  |Rule (p,pl) -> 
   Rule(substitute_in_predicate s p, List.fold_left (fun acc x -> acc @ [substitute_in_predicate s x]) [] pl)
    ;;

module Substitution :
  sig
    type key = variable
    type +'a t
    val empty : 'a t
    val is_empty : 'a t -> bool
    val mem : key -> 'a t -> bool
    val add : key -> 'a -> 'a t -> 'a t
    val update : key -> ('a option -> 'a option) -> 'a t -> 'a t
    val singleton : key -> 'a -> 'a t
    val remove : key -> 'a t -> 'a t
    val merge :
      (key -> 'a option -> 'b option -> 'c option) -> 'a t -> 'b t -> 'c t
    val union : (key -> 'a -> 'a -> 'a option) -> 'a t -> 'a t -> 'a t
    val compare : ('a -> 'a -> int) -> 'a t -> 'a t -> int
    val equal : ('a -> 'a -> bool) -> 'a t -> 'a t -> bool
    val iter : (key -> 'a -> unit) -> 'a t -> unit
    val fold : (key -> 'a -> 'b -> 'b) -> 'a t -> 'b -> 'b
    val for_all : (key -> 'a -> bool) -> 'a t -> bool
    val exists : (key -> 'a -> bool) -> 'a t -> bool
    val filter : (key -> 'a -> bool) -> 'a t -> 'a t
    val partition : (key -> 'a -> bool) -> 'a t -> 'a t * 'a t
    val cardinal : 'a t -> int
    val bindings : 'a 

val string_of_substitution : term Substitution.t -> string = <fun>


val substitute_in_term : term Substitution.t -> term -> term = <fun>


val substitute_in_predicate : term Substitution.t -> predicate -> predicate =
  <fun>


val substitute_in_clause : term Substitution.t -> clause -> clause = <fun>


In [10]:
let s = 
  let open Substitution in 
  add (var "Y") (const_t "0") (add (var "X") (var_t "Y") empty);;

assert (substitute_in_term s (func_t "f" [var_t "X"; var_t "Y"; const_t "a"]) = 
        func_t "f" [var_t "Y"; const_t "0"; const_t "a"]);;
assert (substitute_in_predicate s (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        pred "p" [var_t "Y"; const_t "0"; const_t "a"]);;
assert (substitute_in_clause s (fact @@ pred "p" [var_t "X"; var_t "Y"; const_t "a"]) = 
        (fact @@ pred "p" [var_t "Y"; const_t "0"; const_t "a"]));;
assert (substitute_in_clause s (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]) = 
        (rule (pred "p" [var_t "Y"; const_t "0"; const_t "a"]) [pred "q" [const_t "a"; const_t "b"; const_t "a"]]))

val s : term Substitution.t = <abstr>


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


We define a function `freshen` that given a clause, returns a clause where are the variables have been renamed with fresh variables. 

In [11]:
let counter = ref 0
let fresh () = 
  let c = !counter in
  counter := !counter + 1;
  V (Variable ("_G" ^ string_of_int c))

let freshen c =
  let vars = variables_of_clause c in
  let s = VarSet.fold (fun v s -> Substitution.add v (fresh()) s) vars Substitution.empty in
  substitute_in_clause s c

val counter : int ref = {contents = 0}


val fresh : unit -> term = <fun>


val freshen : clause -> clause = <fun>


For example, 

In [12]:
let c = (rule (pred "p" [var_t "X"; var_t "Y"; const_t "a"]) [pred "q" [var_t "X"; const_t "b"; const_t "a"]])
let _ = string_of_clause c
let _ = string_of_clause (freshen c)

val c : clause =
  Rule
   (Predicate ("p", [V (Variable "X"); V (Variable "Y"); C (Constant "a")]),
   [Predicate ("q", [V (Variable "X"); C (Constant "b"); C (Constant "a")])])


- : string = "p(X, Y, a) :- q(X, b, a)."


- : string = "p(_G0, _G1, a) :- q(_G0, b, a)."


## Problem 4

Implement the function:

```ocaml
mgu : term -> term -> term Substitution.t
```

returns the most general unifier of the given terms. The function should raise the exception `Not_unfifiable`, if the given terms are not unifiable. 

Using `mgu`, implement the function:

```ocaml
mgu_predicate : predicate -> predicate -> term Substitution.t
```

which returns the most general unifier of the given predicates. 

In [13]:
exception Not_unifiable

let rec zip l1 l2 acc= 
  match l1,l2 with
  |_,[] -> acc
  |[],_ -> acc
  |h1::t1,h2::t2 -> zip t1 t2 (acc@[(h1,h2)])


let rec mgu_aux tp1 tp2 kvmap =
  let t1 = substitute_in_term kvmap tp1 in
  let t2 = substitute_in_term kvmap tp2 in
   if t1 = t2 then kvmap else 
  match t1,t2 with
  |V x, _ -> 
    if(occurs_check x t2)then raise Not_unifiable 
    else Substitution.map (fun v -> substitute_in_term (Substitution.add x t2 Substitution.empty) v )  (Substitution.add x t2 kvmap)

  |_,V y -> 
    if(occurs_check y t1)then raise Not_unifiable 
    else Substitution.map (fun v -> substitute_in_term (Substitution.add y t1 Substitution.empty) v )  (Substitution.add y t1 kvmap)

  |C x, C y -> 
    if(x=y)then kvmap 
    else raise Not_unifiable

  |F(Function(fx,lx)),F(Function(fy,ly)) -> 
    if((fx=fy) && (List.length lx = List.length ly))then
      List.fold_left (fun acc (t1,t2) -> (mgu_aux t1 t2 acc)) kvmap (zip lx ly [])
      else raise Not_unifiable

  |_,_ -> raise Not_unifiable

  
 let mgu t1 t2 =  mgu_aux t1 t2 Substitution.empty ;;

let rec mgu_predicate (Predicate (p1,l1)) (Predicate (p2,l2)) =
  if(p1 <> p2 || (List.length l1 <> List.length l2))then raise Not_unifiable
  else
    List.fold_left (fun acc (t1,t2) -> (mgu_aux t1 t2 acc)) Substitution.empty (zip l1 l2 [])

exception Not_unifiable


val zip : 'a list -> 'b list -> ('a * 'b) list -> ('a * 'b) list = <fun>


val mgu_aux : term -> term -> term Substitution.t -> term Substitution.t =
  <fun>


val mgu : term -> term -> term Substitution.t = <fun>


val mgu_predicate : predicate -> predicate -> term Substitution.t = <fun>


In [14]:
assert (string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "X") (var_t "Y")) = "{; Y -> X}");;
assert (string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; X -> Y}" ||
        string_of_substitution (mgu (var_t "Y") (var_t "X")) = "{; Y -> X}");;
assert (mgu (var_t "Y") (var_t "Y") = Substitution.empty);;
assert (mgu (const_t "0") (const_t "0") = Substitution.empty);;
assert (mgu (const_t "0") (var_t "Y") = Substitution.singleton (var "Y") (const_t "0"));;
assert (
  match mgu (const_t "0") (const_t "1") with
  | _ -> false 
  | exception Not_unifiable -> true);;
assert (
  match mgu (func_t "f" [const_t "0"]) (func_t "g" [const_t "1"]) with
  | _ -> false 
  | exception Not_unifiable -> true);;
assert (mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "X") (var_t "Y") ||
        mgu (func_t "f" [var_t "X"]) (func_t "f" [var_t "Y"]) = Substitution.singleton (var "Y") (var_t "X"));;

- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


- : unit = ()


In [15]:
let t1 = F (Function("f", [V (Variable "X"); V (Variable "Y"); V (Variable "Y")]))
let _ = string_of_term t1
let t2 = F (Function("f", [V (Variable "Y"); V (Variable "Z"); V (Variable "a")]))
let _ = string_of_term t2
let u = mgu t1 t2;;
assert (Substitution.cardinal u = 3);;
assert (string_of_substitution u = "{; X -> a; Y -> a; Z -> a}");;


val t1 : term =
  F (Function ("f", [V (Variable "X"); V (Variable "Y"); V (Variable "Y")]))


- : string = "f(X, Y, Y)"


val t2 : term =
  F (Function ("f", [V (Variable "Y"); V (Variable "Z"); V (Variable "a")]))


- : string = "f(Y, Z, a)"


val u : term Substitution.t = <abstr>


- : unit = ()


- : unit = ()


## Problem 5

Implement the function:

```ocaml
query : ?limit:int -> clause list -> predicate list -> predicate list list 
```

where 

* the first optional argument is the `limit` i.e, maximum number of results to be returned. By default, the limit is set to 10 (see code below).
* the second argument is the `program` which is a list of clauses.
* the third argument is the `goal` which is a list of predicates.

The function returns a list of predicate lists (`results`). Each of these results is a instance of the original goal and is a logical consequence of the program. If the given goal is not a logical consequence of the program, then the result is an empty list. See tests cases for expected results.

For the rule and goal order, choose what Prolog does; choose the left-most subgoal for goal order and first rule in the order in which the rules appear in the program for the rule order. 

Hint: Implement a purely functional recursive solution. The backtracking and choice points naturally fall out of the implementation. The reference solution is 17 lines long. 

In [16]:
let query ?(limit=10) program goal =
let substitute_in_predicate_list s pl = 
  List.fold_left (fun acc x -> acc @ [substitute_in_predicate s x]) [] pl
in
let rec solve p resv g ll =
  if(List.length ll >= limit)then ll
  else  
  List.fold_left (
  fun acc c1 -> 
   let c = freshen c1 in
   match c,resv with
   |_,[] ->acc
   
   |Fact f, _ ->
     begin match mgu_predicate (List.hd resv) f with
     |_ -> let th = mgu_predicate (List.hd resv) f in
     if(List.length (List.tl resv) = 0)then 
     acc @ [(substitute_in_predicate_list th g)] 
     else
solve p (substitute_in_predicate_list th (List.tl resv)) (substitute_in_predicate_list th g) acc
     |exception Not_unifiable -> acc
     end
     
          
   |Rule (h,b), _ -> 
     let rnew = b @ (List.tl resv) in
     begin match mgu_predicate (List.hd resv) h with    
     |_-> let th = mgu_predicate (List.hd resv) h in 
     if(List.length rnew = 0)then acc @ [(substitute_in_predicate_list th g)] else 
     solve p (substitute_in_predicate_list th rnew) (substitute_in_predicate_list th g) acc
     |exception Not_unifiable -> acc
     end
   ) ll p
  in 
  solve program goal goal [] 

val query :
  ?limit:int -> clause list -> predicate list -> predicate list list = <fun>


In [17]:
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]]
let _ = print_endline (string_of_program p)

let g = [pred "f" [const_t "a"; const_t "b"]]
let _ = print_endline (string_of_goal g);;

let g' = match query p g with [v] -> v | _ -> failwith "error";;
assert (g' = g);
print_endline (string_of_goal g')

val p : clause list =
  [Fact (Predicate ("f", [C (Constant "a"); C (Constant "b")]))]


f(a, b).



- : unit = ()


val g : predicate list =
  [Predicate ("f", [C (Constant "a"); C (Constant "b")])]


?- f(a, b)


- : unit = ()


val g' : predicate list =
  [Predicate ("f", [C (Constant "a"); C (Constant "b")])]


?- f(a, b)


- : unit = ()


In [18]:
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]]
let _ = print_endline (string_of_program p)

let g = [pred "f" [const_t "a"; const_t "c"]]
let _ = print_endline (string_of_goal g);;

assert (query p g = [])

val p : clause list =
  [Fact (Predicate ("f", [C (Constant "a"); C (Constant "b")]))]


f(a, b).



- : unit = ()


val g : predicate list =
  [Predicate ("f", [C (Constant "a"); C (Constant "c")])]


?- f(a, c)


- : unit = ()


- : unit = ()


In [19]:
let p = [fact @@ pred "f" [const_t "a"; const_t "b"]]
let _ = print_endline (string_of_program p)

let g = [pred "f" [var_t "X"; const_t "b"]]
let _ = print_endline (string_of_goal g);;

let g' = match query p g with [v] -> v | _ -> failwith "error";;
assert (g' = [pred "f" [const_t "a"; const_t "b"]]);
print_endline (string_of_goal g');;

let g = [pred "f" [var_t "X"; var_t "Y"]]
let _ = print_endline (string_of_goal g);;

let g' = match query p g with [v] -> v | _ -> failwith "error";;
assert (g' = [pred "f" [const_t "a"; const_t "b"]]);
print_endline (string_of_goal g')

val p : clause list =
  [Fact (Predicate ("f", [C (Constant "a"); C (Constant "b")]))]


- : unit = ()


f(a, b).



val g : predicate list =
  [Predicate ("f", [V (Variable "X"); C (Constant "b")])]


?- f(X, b)


...

?- f(X, Y)


- : unit = ()


val g' : predicate list =
  [Predicate ("f", [C (Constant "a"); C (Constant "b")])]


?- f(a, b)


- : unit = ()


In [20]:
let ancestor x y = pred "ancestor" [x;y]
let father x y = pred "father" [x;y]
let father_consts x y =  father (C (Constant x)) (C (Constant y))

let f1 = Fact (father_consts "rickard" "ned")
let f2 = Fact (father_consts "ned" "robb")
let r1 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Y")])
let r2 = Rule (ancestor (var_t "X") (var_t "Y"), [father (var_t "X") (var_t "Z"); ancestor (var_t "Z") (var_t "Y")])

let p = [f1;f2;r1;r2]
let g = [ancestor (const_t "rickard") (const_t "ned")]
let _ = print_endline @@ string_of_program p
let _ = print_endline @@ string_of_goal g
let g' = match query p g with [v] -> v | _ -> failwith "error";;
let _ = print_endline @@ string_of_goal g';;
assert (g' = g)

val ancestor : term -> term -> predicate = <fun>


val father : term -> term -> predicate = <fun>


val father_consts : string -> string -> predicate = <fun>


val f1 : clause =
  Fact (Predicate ("father", [C (Constant "rickard"); C (Constant "ned")]))


val f2 : clause =
  Fact (Predicate ("father", [C (Constant "ned"); C (Constant "robb")]))


...

- : unit = ()


val g' : predicate list =
  [Predicate ("ancestor", [C (Constant "rickard"); C (Constant "ned")])]


?- ancestor(rickard, ned)


- : unit = ()


- : unit = ()


In [21]:
(* Tests backtracking *)
let g = [ancestor (const_t "rickard") (const_t "robb")]
let _ = print_endline @@ string_of_goal g
let g' = match query p g with [v] -> v | _ -> failwith "error";;
let _ = print_endline @@ string_of_goal g';;
assert (g' = g)

val g : predicate list =
  [Predicate ("ancestor", [C (Constant "rickard"); C (Constant "robb")])]


?- ancestor(rickard, robb)


- : unit = ()


val g' : predicate list =
  [Predicate ("ancestor", [C (Constant "rickard"); C (Constant "robb")])]


?- ancestor(rickard, robb)


- : unit = ()


- : unit = ()


In [22]:
(* Tests choice points *)
let g = [ancestor (var_t "X") (const_t "robb")]
let _ = print_endline @@ string_of_goal g
let g1,g2 = match query p g with [v1;v2] -> v1,v2 | _ -> failwith "error";;
let _ = print_endline @@ string_of_goal g1;;
let _ = print_endline @@ string_of_goal g2;;
assert (g1 = [ancestor (const_t "ned") (const_t "robb")]);
assert (g2 = [ancestor (const_t "rickard") (const_t "robb")])

val g : predicate list =
  [Predicate ("ancestor", [V (Variable "X"); C (Constant "robb")])]


?- ancestor(X, robb)


- : unit = ()


val g1 : predicate list =
  [Predicate ("ancestor", [C (Constant "ned"); C (Constant "robb")])]
val g2 : predicate list =
  [Predicate ("ancestor", [C (Constant "rickard"); C (Constant "robb")])]


?- ancestor(ned, robb)


- : unit = ()


?- ancestor(rickard, robb)


- : unit = ()


- : unit = ()


In [23]:
(* Tests choice points *)
let nil = const_t "nil"
let cons h t = func_t "cons" [h;t]
let append x y z = pred "append" [x;y;z]
let c1 = fact @@ append nil (var_t "Q") (var_t "Q")
let c2 = rule (append (cons (var_t "H") (var_t "P")) (var_t "Q") (cons (var_t "H") (var_t "R")))
              [append (var_t "P") (var_t "Q") (var_t "R")]
let p = [c1;c2]
let _ = print_endline @@ string_of_program p

let g = [append (var_t "X") (var_t "Y") (cons (const_t "1") (cons (const_t "2") (cons (const_t "3") nil)))]
let _ = print_endline @@ string_of_goal g

let g' = query p g;;
assert (List.length g' = 4);;
let _ = List.iter (fun g -> print_endline @@ string_of_goal g) g'

val nil : term = C (Constant "nil")


val cons : term -> term -> term = <fun>


val append : term -> term -> term -> predicate = <fun>


val c1 : clause =
  Fact
   (Predicate ("append",
     [C (Constant "nil"); V (Variable "Q"); V (Variable "Q")]))


val c2 : clause =
  Rule
   (Predicate ("append",
     [F (Function ("cons", [V (Variable "H"); V (Variable "P")]));
      V (Variable "Q");
      F (Function ("cons", [V (Variable "H"); V (Variable "R")]))]),
   [Predicate ("append",
     [V (Variable "P"); V (Variable "Q"); V (Variable "R")])])


...

- : unit = ()


val g' : predicate list list =
  [[Predicate ("append",
     [C (Constant "nil");
      F
       (Function ("cons",
         [C (Constant "1");
          F
           (Function ("cons",
             [C (Constant "2");
              F (Function ("cons", [C (Constant "3"); C (Constant "nil")]))]))]));
      F
       (Function ("cons",
         [C (Constant "1");
          F
           (Function ("cons",
             [C (Constant "2");
              F (Function ("cons", [C (Constant "3"); C (Constant "nil")]))]))]))])];
   [Predicate ("append",
     [F (Function ("cons", [C (Constant "1"); C (Constant "nil")]));
      F
       (Function ("cons",
         [C (Constant "2");
          F (Function ("cons", [C (Constant "3"); C (Constant "nil")]))]));
      F
       (Function ("cons",
         [C (Constant "1");
          F
           (Function ("cons",
             [C (Constant "2");
              F (Function ("cons", [C (Constant "3"); C (Constant "nil")]))]))]))])];
   [Predicate ("append"

- : unit = ()


?- append(nil, cons(1, cons(2, cons(3, nil))), cons(1, cons(2, cons(3, nil))))
?- append(cons(1, nil), cons(2, cons(3, nil)), cons(1, cons(2, cons(3, nil))))
?- append(cons(1, cons(2, nil)), cons(3, nil), cons(1, cons(2, cons(3, nil))))
?- append(cons(1, cons(2, cons(3, nil))), nil, cons(1, cons(2, cons(3, nil))))


- : unit = ()
